In [ ]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time

import tqdm
import pandas as pd

import re

In [ ]:
# 데이터프레임 생성
naver_df = pd.DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
#options.add_argument('headless') # headless 모드 설정 (창 안 띄움)
options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

#Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome(chrome_path, options=options)

In [ ]:
naver_df = pd.DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])
query = "한라산 여행"

In [ ]:
driver.get('http://blog.naver.com')
soup = BeautifulSoup('http://blog.naver.com')

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_xpath("""//*[@id="header"]/div[1]/div/div[2]/form/fieldset/div/input""")
element.send_keys(query, Keys.ENTER)  # query_txt는 위에서 입력한 '이재용'
#element.send_keys(Keys.ENTER)
time.sleep(1)

#Step 3. 기간 선택
#element_date = driver.find_element_by_link_text("기간 전체")
element_date = driver.find_element_by_css_selector("#content > section > div.category_search > div.search_information > div > div > a")
element_date.send_keys(Keys.ENTER)

#Step 4. 날짜 입력
# 시작 날짜 입력하기
element_start = driver.find_element_by_css_selector("#search_start_date")
element_start.click()
time.sleep(1)
element_start.send_keys('2020-03-01')
# 빈 곳 클릭
driver.find_element_by_css_selector("#content > section > div.category_search > div.search_information > div > div > div > a:nth-child(4)").click()

# 종료 날짜 입력하기
element_end = driver.find_element_by_css_selector("#search_end_date")
element_end.click()
time.sleep(1)
element_end.send_keys('2021-10-01')
# 빈 곳 클릭
driver.find_element_by_css_selector("#content > section > div.category_search > div.search_information > div > div > div > a:nth-child(4)").click()

# Step 5. 날짜 입력 후 "적용" 버튼 클릭
driver.find_element_by_xpath("""//*[@id="periodSearch"]""").click()
time.sleep(0.5)

# 1차 크롤링 (url, 제목) 시작
total_page = 0
btn_cnt = 1
while total_page != 4000:
    # 한 페이지 안의 총 기사 개수
    t_len = len(driver.find_elements_by_class_name("list_search_post"))

    for i in range(1, t_len+1):
        a = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner".format(i))

        # url 수집
        url = a.get_attribute('href')

        # 제목 수집
        title = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span".format(i)).text
        
        naver_df.loc[total_page] = ['nan', '네이버', query, title, 'nan', url, 'nan']

        total_page += 1
        print(total_page)
    
    btn_cnt += 1

    # 다음 버튼 누르기
    if btn_cnt % 10 == 1:
        if btn_cnt > 11:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a.button_next")
            btn.send_keys(Keys.ENTER)
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a")
            btn.send_keys(Keys.ENTER)
        time.sleep(1)
    else:
        if btn_cnt > 11:
            if (btn_cnt % 10) == 0:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child(11) > a")
            else:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format((btn_cnt%10)+1))
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format(btn_cnt))

        btn.send_keys(Keys.ENTER)
        time.sleep(1)


In [ ]:
# 2차 크롤링 (블로그 제목, 포스팅 날짜, 글쓴이) 시작
for i in range(len(naver_df)):
    driver.get(naver_df.loc[i]['url'])
    driver.switch_to.frame('mainFrame')
    overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
    contents = driver.find_elements_by_css_selector(overlays)

    content_list = ""

    for content in contents:
        content_list = content_list + content.text

    # 글쓴이 수집
    author = driver.find_element_by_xpath("""//*[@id="nickNameArea"]""").text
    
    # date 수집
    try:
        date = driver.find_element_by_css_selector(".se_publishDate.pcol2").text
    except:
        date = driver.find_element_by_css_selector(".date.fil5.pcol2._postAddDate").text
    
    naver_df.loc[i][['날짜', '내용', '글쓴이']] = [date, content_list, author]

In [ ]:
import pandas as pd

save_dir = '/Users/yeonghyeon/Documents/Study/TAVE Research/데이터 분석 프로젝트팀/2. Tourism/data'

# 이전 dataframe과 병합하기
naver_final = pd.read_csv(save_dir+'naver_final.csv')
naver_final = pd.concat([naver_final, naver_df])
naver_final.reset_index(drop=True, inplace=True)
naver_final.to_csv(save_dir+'naver_final.csv', index=False)